### Toronto Neighborhoods: an exploration and clustering

*Let's install the libraries we will use today*

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



ModuleNotFoundError: No module named 'folium'

**Our next task is to download the data from Wikipedia and read it into a Pandas dataframe**

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
print('Data downloaded!')
Tneighbors = pd.read_html(results.text)

*Since our data is in a list of DataFrames, we pull out the first one, which is the one we want.*

In [ ]:
Tneighbors = Tneighbors[0]
Tneighbors

**Our next task will be to clean up the data some.** 

*First, let's elimnate the rows that have neither a Borough or a Neighborhood assigned* 

*Here we will create mask of booleans that tells us if a cell in 'Borough' column is "Not assigned"*

In [ ]:
Tneighborsmask = Tneighbors['Borough'] != 'Not assigned'

*By passing the mask to the dataframe, it removes the rows that correspond to **False** in the mask*

In [ ]:
Tneighbors = Tneighbors[Tneighborsmask]
Tneighbors.reset_index(drop=True,inplace=True)

*Now we find the **Neighborhoods** that are are set to "Not assigned" and make those equal to the "Borough" column for that row.*

In [ ]:
for index, row in Tneighbors.iterrows():
    if row['Neighbourhood'] == "Not assigned":
        row['Neighbourhood'] = row['Borough']

**Finally, we group the dataframe by 'Postcode' and 'Borough' and then aggregate the subtables by using a join with ", ".**

In [ ]:
grouped = Tneighbors.groupby(['Postcode', 'Borough'], as_index=False)
neighgroup = grouped.agg({'Neighbourhood':", ".join})
neighgroup = neighgroup.sort_values(['Postcode'], ascending=True)
neighgroup

## Our data frame now is sorted by postal code, and grouped by Borough.

**Let's look at the shape, to make sure we ended up with what we expeced**

In [ ]:
neighgroup.shape